# Exploration des données

In [29]:
import os
import sys
import json5
import json
import pandas as pd

In [2]:
sys.path.append('..')

In [3]:
from transforms import transform
from transforms import concatenate_data

In [4]:
path_data = '..\data'

## Drugs

In [5]:
drugs = pd.read_csv(os.path.join(path_data, 'drugs.csv'))
drugs

,atccode,drug
0,A04AD,DIPHENHYDRAMINE
1,S03AA,TETRACYCLINE
2,V03AB,ETHANOL
3,A03BA,ATROPINE
4,A01AD,EPINEPHRINE
5,6302001,ISOPRENALINE
6,R01AD,BETAMETHASONE


In [6]:
drugs_list = drugs['drug'].str.lower().values.tolist()
print(drugs_list)

['diphenhydramine', 'tetracycline', 'ethanol', 'atropine', 'epinephrine', 'isoprenaline', 'betamethasone']


## Clinical Trials


traiter les cas id=NCT03490942 dupliqué

### Extarct

In [7]:
clinical_trials = pd.read_csv(os.path.join(path_data, 'clinical_trials.csv'))

### Transform

In [8]:
tf_clinical_trials = transform(clinical_trials, drugs_list)

In [57]:
tf_clinical_trials

,id,date,journal,drugs,values
0,NCT01967433,2020-01-01,Journal of emergency nursing,diphenhydramine,{'Journal of emergency nursing': {'2020-01-01'...
1,NCT04189588,2020-01-01,Journal of emergency nursing,diphenhydramine,{'Journal of emergency nursing': {'2020-01-01'...
3,NCT04237091,2020-01-01,Journal of emergency nursing,diphenhydramine,{'Journal of emergency nursing': {'2020-01-01'...
4,NCT04153396,2020-01-01,Hôpitaux Universitaires de Genève,betamethasone,{'Hôpitaux Universitaires de Genève': {'2020-0...
7,NCT04188184,2020-04-27,Journal of emergency nursing,epinephrine,{'Journal of emergency nursing': {'2020-04-27'...


## PubMed CSV

### Extarct

In [9]:
pubmed_csv = pd.read_csv(os.path.join(path_data, 'pubmed.csv'))

### Transform

In [10]:
tf_pubmed_csv = transform(pubmed_csv, drugs_list)#.data_dict.iloc[3]

In [56]:
tf_pubmed_csv

,id,date,journal,drugs,values
1,2,2019-01-01,Journal of emergency nursing,diphenhydramine,{'Journal of emergency nursing': {'2019-01-01'...
2,3,2019-02-01,The Journal of pediatrics,diphenhydramine,{'The Journal of pediatrics': {'2019-02-01': [...
3,4,2020-01-01,Journal of food protection,tetracycline,{'Journal of food protection': {'2020-01-01': ...
4,5,2020-02-01,American journal of veterinary research,tetracycline,{'American journal of veterinary research': {'...
5,6,2020-01-01,Psychopharmacology,ethanol,{'Psychopharmacology': {'2020-01-01': [6]}}
5,6,2020-01-01,Psychopharmacology,tetracycline,{'Psychopharmacology': {'2020-01-01': [6]}}
6,7,2020-01-02,The journal of allergy and clinical immunology...,epinephrine,{'The journal of allergy and clinical immunolo...
7,8,2020-01-03,The journal of allergy and clinical immunology...,epinephrine,{'The journal of allergy and clinical immunolo...


## PubMed JSON

Problème sur le fichier json - cause: "trailing comma"
Mauvaise spécification du fichier JSON  conforméméent à la RFC-7159.

### Extract

In [11]:
with open(os.path.join(path_data, 'pubmed.json')) as f:
    data = json5.load(f)
    pubmed_json = pd.DataFrame(data)

### Transform

In [12]:
tf_pubmed_json = transform(pubmed_json, drugs_list)#.data_dict.iloc[3]

In [ ]:
tf_pubmed_json

### Concatenate data

In [38]:
final_results = concatenate_data([tf_clinical_trials, tf_pubmed_csv, tf_pubmed_json])

In [51]:
json_data = final_results.set_index('drugs').to_dict()['values']
type(json_data)
json_data

{'atropine': {'The journal of maternal-fetal & neonatal medicine': {'2020-01-03': ['']}},
 'betamethasone': {'The journal of maternal-fetal & neonatal medicine': {'2020-01-01': [10],
   '2020-01-03': ['']},
  'Journal of back and musculoskeletal rehabilitation': {'2020-01-01': ['11']},
  'Hôpitaux Universitaires de Genève': {'2020-01-01': ['NCT04153396']}},
 'diphenhydramine': {'Journal of emergency nursing': {'2020-01-01': ['NCT01967433',
    'NCT04189588',
    'NCT04237091'],
   '2019-01-01': [2]},
  'The Journal of pediatrics': {'2019-02-01': [3]}},
 'epinephrine': {'The journal of allergy and clinical immunology. In practice': {'2020-01-03': [8],
   '2020-01-02': [7]},
  'Journal of emergency nursing': {'2020-04-27': ['NCT04188184']}},
 'ethanol': {'Psychopharmacology': {'2020-01-01': [6]}},
 'isoprenaline': {'Journal of photochemistry and photobiology. B, Biology': {'2020-01-01': [9]}},
 'tetracycline': {'American journal of veterinary research': {'2020-02-01': [5]},
  'Psychophar

In [58]:
import json 
# # create json object from dictionary
# json_data = json.dumps(json_data, indent=2)

# open file for writing, "w" 
with open("drugs_map.json","w") as write_file:
    json.dump(json_data, write_file, indent=2)

# close file
f.close()


### Journal mentionnant le plus de médicament

In [61]:
pd.read_json("drugs_map.json").count(axis=1)

The journal of maternal-fetal & neonatal medicine              2
Journal of back and musculoskeletal rehabilitation             1
Hôpitaux Universitaires de Genève                              1
Journal of emergency nursing                                   2
The Journal of pediatrics                                      1
The journal of allergy and clinical immunology. In practice    1
Psychopharmacology                                             2
Journal of photochemistry and photobiology. B, Biology         1
American journal of veterinary research                        1
Journal of food protection                                     1
dtype: int64

In [62]:
pd.read_json("drugs_map.json")

,atropine,betamethasone,diphenhydramine,epinephrine,ethanol,isoprenaline,tetracycline
The journal of maternal-fetal & neonatal medicine,{'2020-01-03': ['']},"{'2020-01-01': [10], '2020-01-03': ['']}",NaN,NaN,NaN,NaN,NaN
Journal of back and musculoskeletal rehabilitation,NaN,{'2020-01-01': ['11']},NaN,NaN,NaN,NaN,NaN
Hôpitaux Universitaires de Genève,NaN,{'2020-01-01': ['NCT04153396']},NaN,NaN,NaN,NaN,NaN
Journal of emergency nursing,NaN,NaN,"{'2020-01-01': ['NCT01967433', 'NCT04189588', ...",{'2020-04-27': ['NCT04188184']},NaN,NaN,NaN
The Journal of pediatrics,NaN,NaN,{'2019-02-01': [3]},NaN,NaN,NaN,NaN
The journal of allergy and clinical immunology. In practice,NaN,NaN,NaN,"{'2020-01-03': [8], '2020-01-02': [7]}",NaN,NaN,NaN
Psychopharmacology,NaN,NaN,NaN,NaN,{'2020-01-01': [6]},NaN,{'2020-01-01': [6]}
"Journal of photochemistry and photobiology. B, Biology",NaN,NaN,NaN,NaN,NaN,{'2020-01-01': [9]},NaN
American journal of veterinary research,NaN,NaN,NaN,NaN,NaN,NaN,{'2020-02-01': [5]}
Journal of food protection,NaN,NaN,NaN,NaN,NaN,NaN,{'2020-01-01': [4]}


In [63]:
pd.read_json("drugs_map.json").count(axis=0)

atropine           1
betamethasone      3
diphenhydramine    2
epinephrine        2
ethanol            1
isoprenaline       1
tetracycline       3
dtype: int64